# KST Course Engine — Demo Notebook

This notebook demonstrates the full pipeline:
1. Parse a course definition
2. Visualize the structure
3. Simulate learner assessment
4. Run optimization algorithms

In [ ]:
from kst_core import (
    parse_file,
    validate_learning_space,
    hasse_mermaid,
    course_json,
    BLIMParameters,
    AdaptiveAssessment,
    simulate_responses,
    LearningModel,
    LearningRate,
    estimate_item_difficulty,
    optimal_teaching_sequence,
)
import numpy as np

## 1. Parse a Course

In [ ]:
course = parse_file("diamond-lattice.kst.yaml")
print(f"Course: {course.name}")
print(f"Items: {len(course.domain)}")
print(f"States: {len(course.states)}")
print(f"Prerequisites: {len(course.prerequisite_graph.edges)}")

## 2. Validate

In [ ]:
report = validate_learning_space(course.domain, course.states)
print(f"Valid: {report.is_valid}")
for r in report.results:
    status = 'PASS' if r.passed else 'FAIL'
    print(f"  [{status}] {r.property_name}: {r.message}")

## 3. Learning Paths

In [ ]:
ls = course.to_learning_space()
paths = ls.learning_paths()
print(f"Total learning paths: {len(paths)}")
for i, path in enumerate(paths):
    print(f"  {i+1}. {' -> '.join(item.id for item in path)}")

## 4. Visualize (Mermaid)

In [ ]:
print(hasse_mermaid(ls))

## 5. Simulate Assessment

In [ ]:
rng = np.random.default_rng(42)
params = BLIMParameters.uniform(course.domain)
state_list = sorted(course.states, key=lambda s: (len(s), sorted(s.item_ids)))

true_state = state_list[3]  # Pick a state
print(f"True state: {sorted(true_state.item_ids)}")

responses = simulate_responses(true_state, params, rng=rng)
print(f"Responses: {responses}")

session = AdaptiveAssessment.start(course.domain, course.states, params)
result = session.run(responses)
print(f"Estimated state: {sorted(result.current_estimate.item_ids)}")
print(f"Correct: {result.current_estimate == true_state}")

## 6. Learning Trajectory Simulation

In [ ]:
rates = LearningRate.uniform(course.domain)
model = LearningModel(space=ls, rates=rates)
traj = model.simulate_trajectory(rng=rng)
print(f"Trajectory length: {len(traj)}")
for i, state in enumerate(traj):
    print(f"  Step {i}: {sorted(state.item_ids) if state.item_ids else '{}'}")

expected = model.expected_steps()
print(f"\nExpected steps from empty: {expected[course.domain.empty_state]:.1f}")

## 7. Optimization

In [ ]:
# Item difficulty
diff_report = estimate_item_difficulty(course.domain, course.prerequisite_graph)
print("Item Difficulty (structural):")
for item in diff_report.items:
    print(f"  {item.item_id}: depth={item.structural_depth}, difficulty={item.combined_difficulty:.3f}")

In [ ]:
# Optimal teaching sequence
plan = optimal_teaching_sequence(ls)
print(f"Optimal Teaching Plan ({plan.total_expected_steps:.0f} steps):")
for i, step in enumerate(plan.steps):
    print(f"  {i+1}. Teach '{step.item_id}' (remaining: {step.expected_remaining:.0f})")

## 8. JSON Export

In [ ]:
import json
print(json.loads(course_json(course))['states']['count'], 'states exported')